# NEURIPS - Open Polymer Prediction 2025
#### Majd Shammout and Khayre Ali (CUDAWOULDASHOULDA)

#### Project Overview
This notebook works to predict five target variables related to physical properties of polymers. 

1) Glass Transition Temperature (`Tg`)
2) Fractional Free Volume (`FFV`)
3) Crystallization Temperature (`Tc`)
4) Density
5) Radius of Gyration (`Rg`)

#### Methodology
Our team leveraged a stacked ensemble of tree-based models, XGBoost & RandomForest, and a pre-trained GNN. The tree-based models are trained on chemical features from libraries like RDKit and molecular fingerprints. The GNN learns representations from the molecular graph structures - particularly relationships that the 2D descriptors would not capture.

The final predication is generated by a meta-model that learns the best way to combine the outputs from the tree-based and GNN models.

# Imports/installs

In [1]:
%%capture
!pip install /kaggle/input/rdkit-2025-3-3-cp311/rdkit-2025.3.3-cp311-cp311-manylinux_2_28_x86_64.whl -q
!pip install /kaggle/input/torch-geometric-2-6-1/torch_geometric-2.6.1-py3-none-any.whl -q
!pip install mordred --no-index --find-links=file:///kaggle/input/mordred-1-2-0-py3-none-any/

import pandas as pd
import numpy as np
import warnings
import os
import json
import joblib
from mordred import Calculator, descriptors
from rdkit import Chem
from rdkit.Chem import Descriptors, MACCSkeys, rdMolDescriptors, rdmolops
from rdkit.Chem.rdFingerprintGenerator import GetMorganGenerator
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import VarianceThreshold
from xgboost import XGBRegressor
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as PyGDataLoader
from torch_geometric.nn import GATConv, global_mean_pool
import networkx as nx

warnings.filterwarnings('ignore')

# Data loading

In [2]:
train = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/train.csv")
test = pd.read_csv("/kaggle/input/neurips-open-polymer-prediction-2025/test.csv")
print(f"Training samples {len(train)}")
print(f"Training samples {len(test)}")
print(f"tg values: {len(train.loc[train['Tg'].notna()])}, missing tg values: {len(train.loc[train['Tg'].isna()])}")
print(f"ffv values: {len(train.loc[train['FFV'].notna()])}, missing ffv values: {len(train.loc[train['FFV'].isna()])}")
print(f"tc values: {len(train.loc[train['Tc'].notna()])}, missing tc values: {len(train.loc[train['Tc'].isna()])}")
print(f"density values: {len(train.loc[train['Density'].notna()])}, missing density values: {len(train.loc[train['Density'].isna()])}")
print(f"rg values: {len(train.loc[train['Density'].notna()])}, missing rg values: {len(train.loc[train['Rg'].isna()])}")

Training samples 7973
Training samples 3
tg values: 511, missing tg values: 7462
ffv values: 7030, missing ffv values: 943
tc values: 737, missing tc values: 7236
density values: 613, missing density values: 7360
rg values: 613, missing rg values: 7359


In [3]:
def clean_and_validate_smiles_vectorized(smiles_series):
    def clean_single(smiles):
        if not isinstance(smiles, str) or len(smiles) == 0:
            return None
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is not None:
                return Chem.MolToSmiles(mol, canonical=True)
        except:
            pass
        return None
    
    return smiles_series.apply(clean_single)

print("\nLoading external datasets...")

def add_extra_data_clean(df_train, df_extra, target):
    n_samples_before = len(df_train[df_train[target].notnull()])
    
    print(f"Processing {len(df_extra)} {target} samples")
    
    df_extra['SMILES'] = clean_and_validate_smiles_vectorized(df_extra['SMILES'])
    
    before_filter = len(df_extra)
    df_extra = df_extra[df_extra['SMILES'].notnull()]
    df_extra = df_extra.dropna(subset=[target])
    after_filter = len(df_extra)
    
    print(f"Valid samples: {after_filter}/{before_filter}")
    
    if len(df_extra) == 0:
        print(f"No valid data for {target}")
        return df_train
    
    df_extra = df_extra.groupby('SMILES', as_index=False)[target].mean()
    
    cross_smiles = set(df_extra['SMILES']) & set(df_train['SMILES'])
    unique_smiles_extra = set(df_extra['SMILES']) - set(df_train['SMILES'])

    for smile in df_train[df_train[target].isnull()]['SMILES'].tolist():
        if smile in cross_smiles:
            df_train.loc[df_train['SMILES']==smile, target] = \
                df_extra[df_extra['SMILES']==smile][target].values[0]
    
    extra_to_add = df_extra[df_extra['SMILES'].isin(unique_smiles_extra)].copy()
    if len(extra_to_add) > 0:
        TARGETS = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
        for col in TARGETS:
            if col not in extra_to_add.columns:
                extra_to_add[col] = np.nan
        extra_to_add = extra_to_add[['SMILES'] + TARGETS]
        df_train = pd.concat([df_train, extra_to_add], axis=0, ignore_index=True)

    n_samples_after = len(df_train[df_train[target].notnull()])
    print(f'{target}: Added {n_samples_after-n_samples_before} samples, {len(unique_smiles_extra)} unique SMILES')
    return df_train

train_extended = train[['SMILES', 'Tg', 'FFV', 'Tc', 'Density', 'Rg']].copy()

try:
    tc_data = pd.read_csv('/kaggle/input/external-polymer-data/Tc_SMILES.csv')
    tc_data = tc_data.rename(columns={'TC_mean': 'Tc'})
    train_extended = add_extra_data_clean(train_extended, tc_data, 'Tc')
    print("Tc data loaded successfully")
except Exception as e:
    print(f"Failed to load Tc data: {str(e)[:100]}")

try:
    tg_density_data = pd.read_csv('/kaggle/input/polymer-tg-density-excerpt/tg_density.csv')
    train_extended = add_extra_data_clean(train_extended, tg_density_data, 'Tg')
    train_extended = add_extra_data_clean(train_extended, tg_density_data, 'Density')
    print("Tg/Density data loaded successfully")
except Exception as e:
    print(f"Failed to load Tg/Density data: {str(e)[:100]}")

try:
    tgss_data = pd.read_csv('/kaggle/input/external-polymer-data/TgSS_enriched_cleaned.csv')
    if 'Tg' in tgss_data.columns:
        train_extended = add_extra_data_clean(train_extended, tgss_data[['SMILES', 'Tg']], 'Tg')
        print("TgSS data loaded successfully")
except Exception as e:
    print(f"Failed to load TgSS data: {str(e)[:100]}")

try:
    jcim_data = pd.read_csv('/kaggle/input/external-polymer-data/JCIM_sup_bigsmiles.csv')
    jcim_data = jcim_data[['SMILES', 'Tg (C)']].rename(columns={'Tg (C)': 'Tg'})
    train_extended = add_extra_data_clean(train_extended, jcim_data, 'Tg')
    print("JCIM data loaded successfully")
except Exception as e:
    print(f"Failed to load JCIM data: {str(e)[:100]}")

try:
    tg_xlsx = pd.read_excel('/kaggle/input/external-polymer-data/data_tg3.xlsx', sheet_name='Лист1')
    tg_xlsx = tg_xlsx.rename(columns={'Tg [K]': 'Tg'})
    tg_xlsx['Tg'] = tg_xlsx['Tg'] - 273.15
    train_extended = add_extra_data_clean(train_extended, tg_xlsx[['SMILES', 'Tg']], 'Tg')
    print("Excel Tg data loaded successfully")
except Exception as e:
    print(f"Failed to load Excel Tg data: {str(e)[:100]}")

try:
    density_data = pd.read_excel('/kaggle/input/external-polymer-data/data_dnst1.xlsx', sheet_name='MatrixRaw')
    density_data = density_data.rename(columns={'density(g/cm3)': 'Density'})
    density_data = density_data[['SMILES', 'Density']]
    density_data = density_data.query('SMILES.notnull() and Density.notnull()')
    density_data['Density'] = pd.to_numeric(density_data['Density'], errors='coerce')
    density_data = density_data.dropna(subset=['Density'])
    density_data['Density'] = density_data['Density'].astype(float) - 0.118
    train_extended = add_extra_data_clean(train_extended, density_data, 'Density')
    print("Density data loaded successfully")
except Exception as e:
    print(f"Failed to load Density data: {str(e)[:100]}")

try:
    dataset4 = pd.read_csv('/kaggle/input/neurips-open-polymer-prediction-2025/train_supplement/dataset4.csv')
    if 'FFV' in dataset4.columns:
        train_extended = add_extra_data_clean(train_extended, dataset4[['SMILES', 'FFV']], 'FFV')
        print("Dataset 4 (FFV) loaded successfully")
except Exception as e:
    print(f"Failed to load Dataset 4: {str(e)[:100]}")

print(f"\nData integration summary:")
print(f"Original samples: {len(train)}")
print(f"Extended samples: {len(train_extended)}")
print(f"Total gain: {len(train_extended) - len(train)} samples")

for target in ['Tg', 'FFV', 'Tc', 'Density', 'Rg']:
    count = train_extended[target].notna().sum()
    original_count = train[target].notna().sum() if target in train.columns else 0
    gain = count - original_count
    print(f"{target}: {count:,} samples (gain: {gain})")

train = train_extended
print("\nData integration complete")


Loading external datasets...
Processing 874 Tc samples
Valid samples: 874/874
Tc: Added 129 samples, 129 unique SMILES
Tc data loaded successfully
Processing 194 Tg samples
Valid samples: 194/194
Tg: Added 90 samples, 57 unique SMILES
Processing 194 Density samples
Valid samples: 181/194
Density: Added 61 samples, 0 unique SMILES
Tg/Density data loaded successfully
Processing 7284 Tg samples
Valid samples: 7284/7284
Tg: Added 7125 samples, 1918 unique SMILES
TgSS data loaded successfully
Processing 662 Tg samples
Valid samples: 662/662
Tg: Added 70 samples, 62 unique SMILES
JCIM data loaded successfully
Processing 501 Tg samples
Valid samples: 501/501
Tg: Added 499 samples, 499 unique SMILES
Excel Tg data loaded successfully
Processing 786 Density samples
Valid samples: 780/786


[19:00:49] SMILES Parse Error: syntax error while parsing: *O[Si](*)([R])[R]
[19:00:49] SMILES Parse Error: check for mistakes around position 12:
[19:00:49] *O[Si](*)([R])[R]
[19:00:49] ~~~~~~~~~~~^
[19:00:49] SMILES Parse Error: Failed parsing SMILES '*O[Si](*)([R])[R]' for input: '*O[Si](*)([R])[R]'
[19:00:49] SMILES Parse Error: syntax error while parsing: *NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4
[19:00:49] SMILES Parse Error: check for mistakes around position 28:
[19:00:49] c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=
[19:00:49] ~~~~~~~~~~~~~~~~~~~~^
[19:00:49] SMILES Parse Error: Failed parsing SMILES '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4' for input: '*NC(=O)c4ccc3c(=O)n(c2ccc([R]c1ccc(*)cc1)cc2)c(=O)c3c4'
[19:00:49] SMILES Parse Error: syntax error while parsing: O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[19:00:49] SMILES Parse Error: check for mistakes around position 7:
[19:00:49] O=C=N[R1]N=C=O.O[R2]O.O[R3]O
[19:00:49] ~~~~~~^
[19:00:49] SMILES Parse Error: F

Density: Added 623 samples, 472 unique SMILES
Density data loaded successfully
Processing 862 FFV samples
Valid samples: 862/862
FFV: Added 862 samples, 272 unique SMILES
Dataset 4 (FFV) loaded successfully

Data integration summary:
Original samples: 7973
Extended samples: 11382
Total gain: 3409 samples
Tg: 8,295 samples (gain: 7784)
FFV: 7,892 samples (gain: 862)
Tc: 866 samples (gain: 129)
Density: 1,297 samples (gain: 684)
Rg: 614 samples (gain: 0)

Data integration complete


In [4]:
train['SMILES'] = clean_and_validate_smiles_vectorized(train['SMILES'])
test['SMILES'] = clean_and_validate_smiles_vectorized(test['SMILES'])

print(f"Problematic smiles to remove from train dataset: {len(train.loc[train['SMILES'].isna()])}")
print(f"Problematic smiles to remove from test dataset: {len(test.loc[test['SMILES'].isna()])}")

train = train.loc[train['SMILES'].notnull()]
test = test.loc[test['SMILES'].notnull()]

Problematic smiles to remove from train dataset: 0
Problematic smiles to remove from test dataset: 0


# Feature engineering

In [5]:
required_descriptors = {'graph_diameter', 'num_cycles', 'avg_shortest_path', 'MolWt', 'LogP', 'TPSA', 'RotatableBonds', 'NumAtoms'}

filters = {
    'Tg': list(set(['NumRotatableBonds', 'NumAromaticRings', 'NumHDonors', 'NumHAcceptors', 'FractionCsp3', 'BertzCT', 'LabuteASA', 'MolWt', 'TPSA', 'MolLogP', 'FractionCsp2', 'HeavyAtomCount', 'NumAromaticCarbocycles', 'NumSaturatedRings', 'ExactMolWt', 'Chi0', 'Chi1', 'Kappa2', 'NumHeteroatoms', 'fr_ester', 'fr_ether', 'fr_amide', 'NumAliphaticRings', 'MaxAbsPartialCharge', 'MinAbsPartialCharge', 'MolMR', 'HallKierAlpha', 'Chi0v', 'Chi1v', 'NumRadicalElectrons', 'NumValenceElectrons', 'fr_benzene', 'fr_C_O', 'fr_C_O_noCOO', 'fr_COO', 'fr_COO2', 'fr_ArN', 'fr_Ar_N', 'fr_Ar_OH', 'BalabanJ', 'Ipc', 'Kappa1', 'Kappa3']).union(required_descriptors)),
    
    'Tc': list(set(['FractionCsp3', 'FractionCsp2', 'NumRotatableBonds', 'NumAromaticRings', 'NumAliphaticRings', 'RingCount', 'MolWt', 'HeavyAtomCount', 'NumHBondDonors', 'NumHBondAcceptors', 'TPSA', 'MolLogP', 'BalabanJ', 'BertzCT', 'HallKierAlpha', 'Kappa1', 'Kappa2', 'Kappa3', 'Phi', 'Chi0', 'Chi1', 'Chi2', 'Chi3', 'Chi4', 'EState_VSA1', 'EState_VSA2', 'EState_VSA3', 'PEOE_VSA1', 'PEOE_VSA2', 'PEOE_VSA3', 'MinPartialCharge', 'MaxPartialCharge', 'MinAbsPartialCharge', 'MaxAbsPartialCharge', 'fr_COO', 'fr_COO2', 'fr_C_O', 'fr_ester', 'fr_ether', 'fr_amide', 'fr_NH0', 'fr_NH1', 'fr_NH2', 'fr_benzene', 'fr_para_hydroxylation', 'NumSaturatedRings', 'NumSaturatedCarbocycles', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'ExactMolWt', 'MolMR', 'LabuteASA', 'Chi0v', 'Chi1v', 'SMR_VSA1', 'SMR_VSA2', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'AvgIpc', 'Ipc', 'MinEStateIndex', 'MaxEStateIndex', 'MinAbsEStateIndex', 'MaxAbsEStateIndex', 'qed', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3']).union(required_descriptors)),
    
    'FFV': list(set(['MolWt', 'ExactMolWt', 'HeavyAtomCount', 'NumRotatableBonds', 'FractionCsp3', 'FractionCsp2', 'FractionCsp', 'LabuteASA', 'TPSA', 'MolLogP', 'MolMR', 'HallKierAlpha', 'Kappa1', 'Kappa2', 'Kappa3', 'Phi', 'Chi0', 'Chi1', 'Chi2', 'Chi3', 'Chi4', 'Chi0v', 'Chi1v', 'Chi2v', 'Chi3v', 'Chi4v', 'BalabanJ', 'BertzCT', 'Ipc', 'MinPartialCharge', 'MaxPartialCharge', 'MinAbsPartialCharge', 'MaxAbsPartialCharge', 'MinEStateIndex', 'MaxEStateIndex', 'PEOE_VSA1', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'SMR_VSA1', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SlogP_VSA1', 'SlogP_VSA2', 'SlogP_VSA3', 'SlogP_VSA4', 'EState_VSA1', 'EState_VSA2', 'EState_VSA3', 'VSA_EState1', 'VSA_EState2', 'VSA_EState3', 'NumHBondDonors', 'NumHBondAcceptors', 'NumAromaticRings', 'NumSaturatedRings', 'NumAliphaticRings', 'RingCount', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'AvgIpc', 'fr_COO', 'fr_COO2', 'fr_C_O', 'fr_C_O_noCOO', 'fr_ester', 'fr_ether', 'fr_Ar_OH', 'fr_phenol', 'fr_benzene', 'fr_Ar_N', 'fr_NH0', 'fr_NH1', 'fr_NH2', 'fr_quatN', 'fr_halogen', 'fr_Al_OH', 'fr_methoxy', 'NumHeteroatoms', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'qed', 'fr_sulfide', 'fr_sulfone', 'fr_sulfonamd', 'fr_nitrile', 'fr_Imine', 'PEOE_VSA9', 'PEOE_VSA10', 'PEOE_VSA11', 'PEOE_VSA12', 'PEOE_VSA13', 'PEOE_VSA14', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA9', 'SMR_VSA10', 'SlogP_VSA5', 'SlogP_VSA6', 'SlogP_VSA7', 'SlogP_VSA8', 'SlogP_VSA9', 'SlogP_VSA10', 'SlogP_VSA11', 'SlogP_VSA12', 'EState_VSA4', 'EState_VSA5', 'EState_VSA6', 'EState_VSA7', 'EState_VSA8', 'EState_VSA9', 'EState_VSA10', 'EState_VSA11', 'VSA_EState4', 'VSA_EState5', 'VSA_EState6', 'VSA_EState7', 'VSA_EState8', 'VSA_EState9', 'VSA_EState10', 'NHOHCount', 'NOCount', 'NumRadicalElectrons', 'NumValenceElectrons']).union(required_descriptors)),
    
    'Density': list(set(['MolWt', 'ExactMolWt', 'HeavyAtomCount', 'NumRotatableBonds', 'FractionCsp3', 'FractionCsp2', 'FractionCsp', 'LabuteASA', 'TPSA', 'MolLogP', 'MolMR', 'HallKierAlpha', 'Kappa1', 'Kappa2', 'Kappa3', 'Phi', 'Chi0', 'Chi1', 'Chi2', 'Chi3', 'Chi4', 'Chi0v', 'Chi1v', 'Chi2v', 'Chi3v', 'Chi4v', 'BalabanJ', 'BertzCT', 'Ipc', 'MinPartialCharge', 'MaxPartialCharge', 'MinAbsPartialCharge', 'MaxAbsPartialCharge', 'PEOE_VSA1', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'SMR_VSA1', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SlogP_VSA1', 'SlogP_VSA2', 'SlogP_VSA3', 'SlogP_VSA4', 'EState_VSA1', 'EState_VSA2', 'EState_VSA3', 'VSA_EState1', 'VSA_EState2', 'VSA_EState3', 'NumHBondDonors', 'NumHBondAcceptors', 'NumAromaticRings', 'NumSaturatedRings', 'NumAliphaticRings', 'RingCount', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'AvgIpc', 'fr_COO', 'fr_COO2', 'fr_C_O', 'fr_C_O_noCOO', 'fr_ester', 'fr_ether', 'fr_amide', 'fr_urea', 'fr_Ar_OH', 'fr_phenol', 'fr_benzene', 'fr_Ar_N', 'fr_NH0', 'fr_NH1', 'fr_NH2', 'fr_halogen', 'fr_Al_OH', 'fr_methoxy', 'fr_sulfide', 'fr_sulfone', 'NumHeteroatoms', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'qed', 'Chi0n', 'Chi1n', 'Chi2n', 'Chi3n', 'Chi4n', 'MinEStateIndex', 'MaxEStateIndex', 'PEOE_VSA7', 'PEOE_VSA8', 'PEOE_VSA9', 'PEOE_VSA10', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA8', 'SMR_VSA9', 'SMR_VSA10', 'SlogP_VSA5', 'SlogP_VSA6', 'SlogP_VSA7', 'SlogP_VSA8', 'SlogP_VSA9', 'SlogP_VSA10', 'SlogP_VSA11', 'SlogP_VSA12', 'EState_VSA4', 'EState_VSA5', 'EState_VSA6', 'EState_VSA7', 'EState_VSA8', 'EState_VSA9', 'EState_VSA10', 'EState_VSA11', 'VSA_EState4', 'VSA_EState5', 'VSA_EState6', 'VSA_EState7', 'VSA_EState8', 'VSA_EState9', 'VSA_EState10', 'NumAromaticCarbocycles', 'NumAromaticHeterocycles', 'NumSaturatedCarbocycles', 'NumSaturatedHeterocycles', 'NumAliphaticCarbocycles', 'NumAliphaticHeterocycles']).union(required_descriptors)),
    
    'Rg': list(set(['MolWt', 'ExactMolWt', 'HeavyAtomCount', 'NumRotatableBonds', 'FractionCsp3', 'FractionCsp2', 'FractionCsp', 'LabuteASA', 'TPSA', 'MolLogP', 'MolMR', 'HallKierAlpha', 'Kappa1', 'Kappa2', 'Kappa3', 'Phi', 'Chi0', 'Chi1', 'Chi2', 'Chi3', 'Chi4', 'Chi0n', 'Chi1n', 'Chi2n', 'Chi3n', 'Chi4n', 'Chi0v', 'Chi1v', 'Chi2v', 'Chi3v', 'Chi4v', 'BalabanJ', 'BertzCT', 'Ipc', 'AvgIpc', 'MinPartialCharge', 'MaxPartialCharge', 'MinAbsPartialCharge', 'MaxAbsPartialCharge', 'MinEStateIndex', 'MaxEStateIndex', 'PEOE_VSA1', 'PEOE_VSA2', 'PEOE_VSA3', 'PEOE_VSA4', 'PEOE_VSA5', 'PEOE_VSA6', 'PEOE_VSA7', 'PEOE_VSA8', 'SMR_VSA1', 'SMR_VSA2', 'SMR_VSA3', 'SMR_VSA4', 'SMR_VSA5', 'SMR_VSA6', 'SMR_VSA7', 'SMR_VSA8', 'SlogP_VSA1', 'SlogP_VSA2', 'SlogP_VSA3', 'SlogP_VSA4', 'SlogP_VSA5', 'SlogP_VSA6', 'EState_VSA1', 'EState_VSA2', 'EState_VSA3', 'EState_VSA4', 'EState_VSA5', 'EState_VSA6', 'VSA_EState1', 'VSA_EState2', 'VSA_EState3', 'VSA_EState4', 'VSA_EState5', 'VSA_EState6', 'NumHBondDonors', 'NumHBondAcceptors', 'NumAromaticRings', 'NumSaturatedRings', 'NumAliphaticRings', 'RingCount', 'BCUT2D_MWHI', 'BCUT2D_MWLOW', 'BCUT2D_CHGHI', 'BCUT2D_CHGLO', 'BCUT2D_LOGPHI', 'BCUT2D_LOGPLOW', 'BCUT2D_MRHI', 'BCUT2D_MRLOW', 'fr_COO', 'fr_COO2', 'fr_C_O', 'fr_C_O_noCOO', 'fr_ester', 'fr_ether', 'fr_amide', 'fr_urea', 'fr_Ar_OH', 'fr_phenol', 'fr_benzene', 'fr_Ar_N', 'fr_NH0', 'fr_NH1', 'fr_NH2', 'fr_halogen', 'fr_Al_OH', 'fr_methoxy', 'fr_sulfide', 'fr_sulfone', 'NumHeteroatoms', 'FpDensityMorgan1', 'FpDensityMorgan2', 'FpDensityMorgan3', 'qed', 'NHOHCount', 'NOCount', 'fr_nitrile', 'fr_azide', 'fr_alkyne']).union(required_descriptors))
}

for key in filters:
    filters[key] = list(set(filters[key]))

In [6]:
def separate_subtables(train_df):
    labels = ['Tg', 'FFV', 'Tc', 'Density', 'Rg']
    subtables = {}
    for label in labels:
        subtables[label] = train_df[['SMILES', label]][train_df[label].notna()]
    return subtables

def augment_smiles_dataset(smiles_list, labels, num_augments=3):
    augmented_smiles = []
    augmented_labels = []
    for smiles, label in zip(smiles_list, labels):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue
        augmented_smiles.append(smiles)
        augmented_labels.append(label)
        for _ in range(num_augments):
            rand_smiles = Chem.MolToSmiles(mol, doRandom=True)
            augmented_smiles.append(rand_smiles)
            augmented_labels.append(label)
    return augmented_smiles, np.array(augmented_labels)

In [7]:
from mordred import Calculator, descriptors

def calculate_mordred_descriptors(mol):
    """Calculate Mordred descriptors for a molecule."""
    calc = Calculator(descriptors, ignore_3D=True)
    result = calc(mol)
    
    mordred_dict = {}
    for desc_name, desc_value in result.items():
        try:
            if isinstance(desc_value, (int, float)) and np.isfinite(desc_value):
                mordred_dict[f'mordred_{desc_name}'] = desc_value
            else:
                mordred_dict[f'mordred_{desc_name}'] = None
        except:
            mordred_dict[f'mordred_{desc_name}'] = None
    
    return mordred_dict

def smiles_to_combined_fingerprints_with_descriptors_batch(smiles_list, selected_descriptors, 
                                                           radius=2, n_bits=128, use_mordred=True):
    """
    Enhanced version with optional Mordred descriptors
    """
    generator = GetMorganGenerator(radius=radius, fpSize=n_bits)
    descriptor_functions = {name: func for name, func in Descriptors.descList if name in selected_descriptors}
    
    fingerprints = []
    descriptors = []
    valid_smiles = []
    invalid_indices = []
    
    mol_cache = {}
    for smiles in set(smiles_list):
        mol_cache[smiles] = Chem.MolFromSmiles(smiles)
    
    if use_mordred:
        mordred_calc = Calculator(descriptors, ignore_3D=True)
    
    for i, smiles in enumerate(smiles_list):
        mol = mol_cache.get(smiles)
        if mol:
            morgan_fp = generator.GetFingerprint(mol)
            maccs_fp = MACCSkeys.GenMACCSKeys(mol)
            combined_fp = np.concatenate([np.array(morgan_fp), np.array(maccs_fp)])
            fingerprints.append(combined_fp)
            
            descriptor_values = {}
            
            for name, func in descriptor_functions.items():
                try:
                    descriptor_values[name] = func(mol)
                except:
                    descriptor_values[name] = None
            
            try:
                descriptor_values['MolWt'] = Descriptors.MolWt(mol)
            except:
                pass
            
            try:
                descriptor_values['LogP'] = Descriptors.MolLogP(mol)
            except:
                pass
            
            try:
                descriptor_values['TPSA'] = rdMolDescriptors.CalcTPSA(mol)
            except:
                pass
            
            try:
                descriptor_values['RotatableBonds'] = Descriptors.NumRotatableBonds(mol)
            except:
                pass
            
            try:
                descriptor_values['NumAtoms'] = mol.GetNumAtoms()
            except:
                pass
            
            try:
                adj = rdmolops.GetAdjacencyMatrix(mol)
                G = nx.from_numpy_array(adj)
                if nx.is_connected(G):
                    descriptor_values['graph_diameter'] = nx.diameter(G)
                    descriptor_values['avg_shortest_path'] = nx.average_shortest_path_length(G)
                else:
                    descriptor_values['graph_diameter'] = 0
                    descriptor_values['avg_shortest_path'] = 0
                descriptor_values['num_cycles'] = len(list(nx.cycle_basis(G)))
            except:
                descriptor_values['graph_diameter'] = None
                descriptor_values['avg_shortest_path'] = None
                descriptor_values['num_cycles'] = None
            
            if use_mordred:
                try:
                    mordred_result = mordred_calc(mol)
                    for desc_name, desc_value in mordred_result.items():
                        try:
                            if isinstance(desc_value, (int, float)) and np.isfinite(desc_value):
                                descriptor_values[f'mordred_{desc_name}'] = desc_value
                            else:
                                descriptor_values[f'mordred_{desc_name}'] = None
                        except:
                            descriptor_values[f'mordred_{desc_name}'] = None
                except:
                    pass
            
            descriptor_values['SMILES'] = smiles
            descriptors.append(descriptor_values)
            valid_smiles.append(smiles)
        else:
            fingerprints.append(np.zeros(n_bits + 167))
            descriptors.append(None)
            valid_smiles.append(None)
            invalid_indices.append(i)
    
    return np.array(fingerprints), descriptors, valid_smiles, invalid_indices

mordred_important = [
    'mordred_nBonds', 'mordred_nBondsO', 'mordred_nBondsS', 'mordred_nBondsD',
    'mordred_nBondsT', 'mordred_nBondsA', 'mordred_nBondsM', 'mordred_nBondsKS',
    'mordred_GATS1c', 'mordred_GATS2c', 'mordred_GATS3c', 'mordred_GATS4c',
    'mordred_MATS1c', 'mordred_MATS2c', 'mordred_MATS3c', 'mordred_MATS4c',
    'mordred_SpMax_D', 'mordred_SpMin_D', 'mordred_SpMax_A', 'mordred_SpMin_A',
    'mordred_EState_VSA1', 'mordred_EState_VSA2', 'mordred_EState_VSA3',
    'mordred_VSA_EState1', 'mordred_VSA_EState2', 'mordred_VSA_EState3',
    'mordred_PEOE_VSA1', 'mordred_PEOE_VSA2', 'mordred_PEOE_VSA3',
    'mordred_SMR_VSA1', 'mordred_SMR_VSA2', 'mordred_SMR_VSA3',
    'mordred_SlogP_VSA1', 'mordred_SlogP_VSA2', 'mordred_SlogP_VSA3',
    'mordred_MPC2', 'mordred_MPC3', 'mordred_MPC4', 'mordred_MPC5',
    'mordred_AMID_C', 'mordred_AMID_N', 'mordred_AMID_O', 'mordred_AMID_X',
    'mordred_TopoPSA', 'mordred_LabuteASA', 'mordred_TPSA',
    'mordred_Diameter', 'mordred_Radius', 'mordred_TopoShapeIndex',
    'mordred_PetitjeanIndex', 'mordred_GeomDiameter', 'mordred_GeomRadius'
]

for key in filters:
    filters[key] = list(set(filters[key] + mordred_important))

subtables = separate_subtables(train)
test_smiles = test['SMILES'].tolist()
test_ids = test['id'].values if 'id' in test.columns else range(len(test))

TARGET_VARIABLES = ["Tg", "FFV", "Tc", "Density", "Rg"]
labels = TARGET_VARIABLES

output_df = pd.DataFrame({'id': test_ids})

data_per_label = {}
test_data_per_label = {}

USE_MORDRED = True

for label in labels:
    print(f"Processing label: {label}")
    print(subtables[label].head())
    print(subtables[label].shape)
    
    original_smiles = subtables[label]['SMILES'].tolist()
    original_labels = subtables[label][label].values
    
    original_smiles, original_labels = augment_smiles_dataset(original_smiles, original_labels, num_augments=1)
    
    fingerprints, descriptors, valid_smiles, invalid_indices = smiles_to_combined_fingerprints_with_descriptors_batch(
        original_smiles, filters[label], radius=2, n_bits=128, use_mordred=USE_MORDRED)
    
    X = pd.DataFrame(descriptors)
    y = np.delete(original_labels, invalid_indices)
    
    available_cols = [col for col in filters[label] if col in X.columns]
    X = X[available_cols]
    
    fp_df = pd.DataFrame(fingerprints, columns=[f'FP_{i}' for i in range(fingerprints.shape[1])])
    
    fp_df.reset_index(drop=True, inplace=True)
    X.reset_index(drop=True, inplace=True)
    X = pd.concat([X, fp_df], axis=1)
    
    training_columns = X.columns.tolist()
    
    print(f"After concat (with Mordred): {X.shape}")
    
    X = X.replace([np.inf, -np.inf], np.nan)
    X = X.fillna(0)
    
    threshold = 0.01
    selector = VarianceThreshold(threshold=threshold)
    X_transformed = selector.fit_transform(X)
    final_columns = selector.get_feature_names_out()
    X = pd.DataFrame(X_transformed, columns=final_columns)
    
    print(f"After variance cut: {X.shape}")
    
    data_per_label[label] = pd.concat([X, pd.Series(y, name='Target'), pd.Series(valid_smiles, name='SMILES')], axis=1)
    print(f"Data stored for {label} with shape: {data_per_label[label].shape}")
    
    fingerprints, descriptors, valid_smiles_test, invalid_indices = smiles_to_combined_fingerprints_with_descriptors_batch(
        test_smiles, filters[label], radius=2, n_bits=128, use_mordred=USE_MORDRED)
    
    test_X = pd.DataFrame(descriptors)
    available_test_cols = [col for col in available_cols if col in test_X.columns]
    test_X = test_X[available_test_cols]
    
    fp_df = pd.DataFrame(fingerprints, columns=[f'FP_{i}' for i in range(fingerprints.shape[1])])
    fp_df.reset_index(drop=True, inplace=True)
    test_X.reset_index(drop=True, inplace=True)
    test_X = pd.concat([test_X, fp_df], axis=1)
    
    test_X = test_X.reindex(columns=training_columns, fill_value=0)

    test_X = test_X.replace([np.inf, -np.inf], np.nan)
    test_X = test_X.fillna(0)

    test_X_transformed = selector.transform(test_X)
    test_X = pd.DataFrame(test_X_transformed, columns=final_columns)
    
    print(f"Test shape: {test_X.shape}")
    test_data_per_label[label] = test_X

for label in labels:
    if 'Target' in data_per_label[label].columns:
        data_per_label[label].rename(columns={'Target': label}, inplace=True)

Processing label: Tg
                                              SMILES     Tg
0                         *CC(*)c1ccccc1C(=O)OCCCCCC   45.0
4  *Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...   34.0
5               *OC(=O)CCCCCCCCC(=O)OC1COC2C(*)COC12    3.0
7  *C(=O)Nc1ccc(Oc2ccc(Oc3ccc(NC(=O)c4ccc5c(c4)C(...  275.0
8  *CC(*)(C)C(=O)OCCCCCCCCCOc1ccc2cc(C(=O)Oc3cccc...   18.0
(8295, 2)
After concat (with Mordred): (16590, 342)
After variance cut: (16590, 303)
Data stored for Tg with shape: (16590, 305)
Test shape: (3, 303)
Processing label: FFV
                                              SMILES       FFV
0                         *CC(*)c1ccccc1C(=O)OCCCCCC  0.374645
1  *Nc1ccc([C@H](CCC)c2ccc(C3(c4ccc([C@@H](CCC)c5...  0.370410
2  *Oc1ccc(S(=O)(=O)c2ccc(Oc3ccc(C4(c5ccc(Oc6ccc(...  0.378860
3  *Nc1ccc(-c2c(-c3ccc(C)cc3)c(-c3ccc(C)cc3)c(N*)...  0.387324
4  *Oc1ccc(OC(=O)c2cc(OCCCCCCCCCOCC3CCCN3c3ccc([N...  0.355470
(7892, 2)
After concat (with Mordred): (15784, 431)
After variance cu

# Train Model

In [8]:
ATOM_MAP = {
    'C': 0, 'N': 1, 'O': 2, 'F': 3, 'P': 4, 'S': 5, 'Cl': 6, 'Br': 7, 'I': 8, 'H': 9,
    'Si': 10, 'Na': 11, '*': 12, 'B': 13, 'Ge': 14, 'Sn': 15, 'Se': 16,
    'Te': 17, 'Ca': 18, 'Cd': 19
}
ATOM_MAP_LEN = len(ATOM_MAP)

LABEL_SPECIFIC_FEATURES = {
    'Tg': ["HallKierAlpha", "MolLogP", "NumRotatableBonds", "TPSA"],
    'FFV': ["NHOHCount", "NumRotatableBonds", "MolWt", "TPSA"],
    'Tc': ["MolLogP", "NumValenceElectrons", "SPS", "MolWt"],
    'Density': ["MolWt", "MolMR", "FractionCSP3", "NumHeteroatoms"],
    'Rg': ["HallKierAlpha", "MolWt", "NumValenceElectrons", "qed"]
}

RDKIT_DESC_CALCULATORS = {name: func for name, func in Descriptors.descList}
RDKIT_DESC_CALCULATORS['qed'] = Descriptors.qed

def smiles_to_graph_label_specific(smiles_str: str, label: str, y_val=None):
    try:
        mol = Chem.MolFromSmiles(smiles_str)
        if mol is None: return None

        global_features = []
        features_to_calculate = LABEL_SPECIFIC_FEATURES.get(label, [])
        for name in features_to_calculate:
            calculator = RDKIT_DESC_CALCULATORS.get(name)
            if calculator:
                try:
                    val = calculator(mol)
                    global_features.append(val if np.isfinite(val) else 0.0)
                except: global_features.append(0.0)
            else: global_features.append(0.0)

        node_features = []
        for atom in mol.GetAtoms():
            atom_f = [0] * ATOM_MAP_LEN
            if atom.GetSymbol() in ATOM_MAP: atom_f[ATOM_MAP[atom.GetSymbol()]] = 1
            atom_f.extend([
                atom.GetAtomicNum(), atom.GetTotalDegree(), atom.GetFormalCharge(),
                atom.GetTotalNumHs(), int(atom.GetIsAromatic())
            ])
            node_features.append(atom_f)
        if not node_features: return None
        x = torch.tensor(node_features, dtype=torch.float)

        edge_indices, edge_attrs = [], []
        for bond in mol.GetBonds():
            i, j = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
            edge_indices.extend([(i, j), (j, i)])
            bond_type = bond.GetBondTypeAsDouble()
            edge_attrs.extend([[bond_type], [bond_type]])
        edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
        edge_attr = torch.tensor(edge_attrs, dtype=torch.float) if edge_attrs else torch.empty((0, 1), dtype=torch.float)

        data_obj = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
        data_obj.u = torch.tensor([global_features], dtype=torch.float)
        if y_val is not None: data_obj.y = torch.tensor([[y_val]], dtype=torch.float)
        return data_obj
    except: return None

def create_dynamic_mlp(input_dim, layer_list, dropout_list):
    layers = []
    current_dim = input_dim
    for neurons, dropout in zip(layer_list, dropout_list):
        layers.extend([torch.nn.Linear(current_dim, neurons), torch.nn.ReLU(), torch.nn.Dropout(dropout)])
        current_dim = neurons
    layers.append(torch.nn.Linear(current_dim, 1))
    return torch.nn.Sequential(*layers)

class TaskSpecificGNN(torch.nn.Module):
    def __init__(self, num_node_features, num_edge_features, num_global_features,
                 hidden_channels_gnn, mlp_neurons, mlp_dropouts, heads=8):
        super().__init__()
        self.convs = torch.nn.ModuleList([
            GATConv(num_node_features, hidden_channels_gnn, heads=heads, edge_dim=num_edge_features),
            GATConv(hidden_channels_gnn * heads, hidden_channels_gnn * 2, heads=heads, edge_dim=num_edge_features),
            GATConv(hidden_channels_gnn * 2 * heads, hidden_channels_gnn * 4, heads=heads, concat=False, edge_dim=num_edge_features)
        ])
        combined_feature_size = (hidden_channels_gnn * 4) + num_global_features
        self.readout_mlp = create_dynamic_mlp(combined_feature_size, mlp_neurons, mlp_dropouts)
        self.config_args = {k: v for k, v in locals().items() if k not in ['self', '__class__']}

    def forward(self, data):
        x, edge_index, edge_attr, u, batch = data.x, data.edge_index, data.edge_attr, data.u, data.batch
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index, edge_attr))
            if i < len(self.convs) - 1: x = F.dropout(x, p=0.5, training=self.training)
        graph_embedding = global_mean_pool(x, batch)
        combined_features = torch.cat([graph_embedding, u], dim=1)
        return self.readout_mlp(combined_features)

def load_gnn_model(label, model_dir):
    model_path = os.path.join(model_dir, f"gnn_model_{label}.pth")
    config_path = os.path.join(model_dir, f"gnn_config_{label}.json")
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    if not (os.path.exists(model_path) and os.path.exists(config_path)): return None
    with open(config_path, 'r') as f: config = json.load(f)
    try:
        model = TaskSpecificGNN(**config).to(DEVICE)
        model.load_state_dict(torch.load(model_path, map_location=DEVICE))
        model.eval()
        return model
    except Exception as e:
        print(f"Error loading model for {label}: {e}")
        return None

def scale_graph_features(data_list, u_scaler, x_scaler, atom_map_len):
    for data in data_list:
        data.u = torch.tensor(u_scaler.transform(data.u.numpy()), dtype=torch.float)
        x_one_hot, x_continuous = data.x[:, :atom_map_len], data.x[:, atom_map_len:]
        x_continuous_scaled = torch.tensor(x_scaler.transform(x_continuous.numpy()), dtype=torch.float)
        data.x = torch.cat([x_one_hot, x_continuous_scaled], dim=1)
    return data_list

def predict_with_gnn(model, smiles_list, label, u_scaler, x_scaler, atom_map_len):
    if model is None or u_scaler is None or x_scaler is None: return np.full(len(smiles_list), np.nan)
    DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    test_data_raw = [smiles_to_graph_label_specific(s, label) for s in smiles_list]
    valid_indices = [i for i, d in enumerate(test_data_raw) if d is not None]
    valid_data = [d for d in test_data_raw if d is not None]
    if not valid_data: return np.full(len(smiles_list), np.nan)
    
    valid_data_scaled = scale_graph_features(valid_data, u_scaler, x_scaler, atom_map_len)
    loader = PyGDataLoader(valid_data_scaled, batch_size=32, shuffle=False)
    
    model.eval()
    preds = []
    with torch.no_grad():
        for data in loader:
            preds.append(model(data.to(DEVICE)).cpu())
    
    scaled_preds = torch.cat(preds, dim=0).numpy().flatten()
    final_preds = np.full(len(smiles_list), np.nan)
    if len(scaled_preds) == len(valid_indices): final_preds[valid_indices] = scaled_preds
    return final_preds

In [9]:
def run_gnn_predictions(test_smiles_list, test_ids_list):
    """Loads all pre-trained GNN models and scalers to generate predictions for the test set."""
    gnn_preds_df = pd.DataFrame({'id': test_ids_list})
    model_base_dir = '/kaggle/input/neurips-2025/GATConv_v29/models/gnn/'
    
    mlp_configs = {
        "Tg": {"neurons": [512, 256, 128], "dropouts": [0.5, 0.4, 0.2]},
        "Density": {"neurons": [1024, 256, 64], "dropouts": [0.5, 0.4, 0.3]},
        "FFV": {"neurons": [1024, 512, 64], "dropouts": [0.6, 0.5, 0.4]},
        "Tc": {"neurons": [128, 64], "dropouts": [0.4, 0.3]},
        "Rg": {"neurons": [128, 64, 64], "dropouts": [0.4, 0.3, 0.3]},
    }

    for label in TARGET_VARIABLES:
        print(f"\\n{'='*20} Processing GNN for label: {label} {'='*20}")
        try:
            y_scaler = joblib.load(f'{model_base_dir}gnn_yscaler_{label}.joblib')
            u_scaler = joblib.load(f'{model_base_dir}gnn_uscaler_{label}.joblib')
            x_scaler = joblib.load(f'{model_base_dir}gnn_xscaler_{label}.joblib')
        except FileNotFoundError:
            print(f"CRITICAL: Scaler files not found for {label}. Cannot make predictions.")
            gnn_preds_df[label] = 0.0 # Fallback
            continue

        ensemble_models = []
        for fold in range(10): # There are 10 pre-trained models per target
            model = load_gnn_model(f"{label}_fold{fold}", model_base_dir)
            if model: ensemble_models.append(model)
        
        if not ensemble_models:
            print(f"CRITICAL: No models loaded for {label}.")
            gnn_preds_df[label] = y_scaler.inverse_transform([[0.0]])[0][0] # Fallback to median
            continue

        print(f"Loaded {len(ensemble_models)} models for {label} ensemble.")
        
        all_fold_preds_scaled = [
            predict_with_gnn(model, test_smiles_list, label, u_scaler, x_scaler, ATOM_MAP_LEN)
            for model in ensemble_models
        ]
        
        final_preds_scaled = np.nanmean(np.stack(all_fold_preds_scaled), axis=0)
        filled_preds_scaled = pd.Series(final_preds_scaled).fillna(0.0) # Impute with scaled median (0.0 for RobustScaler)
        
        final_preds_original = y_scaler.inverse_transform(filled_preds_scaled.values.reshape(-1, 1)).flatten()
        gnn_preds_df[label] = final_preds_original

    return gnn_preds_df

gnn_predictions_df = run_gnn_predictions(test_smiles, test_ids)
print("\\nGNN Predictions Complete:")
print(gnn_predictions_df.head())

\n==================== Processing GNN for label: Tg ====================
Loaded 10 models for Tg ensemble.
\n==================== Processing GNN for label: FFV ====================
Loaded 10 models for FFV ensemble.
\n==================== Processing GNN for label: Tc ====================
Loaded 10 models for Tc ensemble.
\n==================== Processing GNN for label: Density ====================
Loaded 10 models for Density ensemble.
\n==================== Processing GNN for label: Rg ====================
Loaded 10 models for Rg ensemble.
\nGNN Predictions Complete:
           id          Tg       FFV        Tc   Density         Rg
0  1109053969  189.751849  0.373679  0.187229  1.168659  21.798934
1  1422188626  174.595008  0.375173  0.266134  1.103980  23.134463
2  2032016830  111.029738  0.350787  0.250217  1.113623  19.598631


In [10]:
TARGET_VARIABLES = ["Tg", "FFV", "Tc", "Density", "Rg"]
RANDOM_STATE = 42
N_FOLDS = 5

predictions_df = pd.DataFrame({'id': test_ids})
mae_scores = {}
oof_data = {}

for target in TARGET_VARIABLES:
    print(f"\nTraining for {target}...")

    X_test = test_data_per_label[target]
    
    full_data = data_per_label[target].dropna(subset=[target])
    y = full_data[target]
    X = full_data.drop(columns=[target, 'SMILES'])
    smiles_for_oof = full_data['SMILES']

    X_features = X.to_numpy()
    y_values = y.to_numpy()
    X_test_array = X_test.to_numpy()
    
    X_features = np.nan_to_num(X_features, nan=0.0, posinf=0.0, neginf=0.0)
    X_test_array = np.nan_to_num(X_test_array, nan=0.0, posinf=0.0, neginf=0.0)
    X_features = np.clip(X_features, -1e10, 1e10)
    X_test_array = np.clip(X_test_array, -1e10, 1e10)
    
    print(f"Data shape: {X_features.shape}, features: {X.shape[1]}")
    
    kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)
    
    oof_predictions = np.zeros(len(y_values))
    test_predictions = np.zeros((N_FOLDS, len(X_test_array)))
    fold_scores = []
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_features)):
        print(f"  Fold {fold + 1}/{N_FOLDS}")
        
        X_train_fold, X_val_fold = X_features[train_idx], X_features[val_idx]
        y_train_fold, y_val_fold = y_values[train_idx], y_values[val_idx]
        
        if target == "Tg":
            xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=8, reg_lambda=2.3, subsample=0.7, colsample_bytree=0.7, random_state=RANDOM_STATE, tree_method='hist', early_stopping_rounds=50)
            rf_model = RandomForestRegressor(n_estimators=200, max_depth=15, min_samples_leaf=2, max_features=0.7, random_state=RANDOM_STATE, n_jobs=-1)
        elif target == "Rg":
            xgb_model = XGBRegressor(n_estimators=300, learning_rate=0.08, max_depth=5, reg_lambda=1.0, random_state=RANDOM_STATE, tree_method='hist', early_stopping_rounds=30)
            rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
        elif target == "FFV":
            xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.08, max_depth=4, reg_lambda=3.0, random_state=RANDOM_STATE, tree_method='hist', early_stopping_rounds=50)
            rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
        elif target == "Tc":
            xgb_model = XGBRegressor(n_estimators=400, learning_rate=0.05, max_depth=5, reg_lambda=10.0, random_state=RANDOM_STATE, tree_method='hist', early_stopping_rounds=40)
            rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)
        elif target == "Density":
            xgb_model = XGBRegressor(n_estimators=500, learning_rate=0.1, max_depth=5, reg_lambda=3.0, random_state=RANDOM_STATE, tree_method='hist', early_stopping_rounds=50)
            rf_model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=RANDOM_STATE, n_jobs=-1)

        xgb_model.fit(X_train_fold, y_train_fold, eval_set=[(X_val_fold, y_val_fold)], verbose=False)
        xgb_oof_preds = xgb_model.predict(X_val_fold)
        xgb_test_preds = xgb_model.predict(X_test_array)
        
        rf_model.fit(X_train_fold, y_train_fold)
        rf_oof_preds = rf_model.predict(X_val_fold)
        rf_test_preds = rf_model.predict(X_test_array)
        
        fold_oof_preds = (xgb_oof_preds + rf_oof_preds) / 2
        fold_test_preds = (xgb_test_preds + rf_test_preds) / 2
        
        oof_predictions[val_idx] = fold_oof_preds
        test_predictions[fold] = fold_test_preds
        
        fold_mae = mean_absolute_error(y_val_fold, fold_oof_preds)
        fold_scores.append(fold_mae)
        print(f"    Fold {fold + 1} MAE: {fold_mae:.4f}")
    
    cv_mae = mean_absolute_error(y_values, oof_predictions)
    mae_scores[target] = cv_mae
    print(f"  CV MAE for {target}: {cv_mae:.4f} ± {np.std(fold_scores):.4f}")
    
    predictions_df[target] = np.mean(test_predictions, axis=0)
    
    oof_data[target] = {
        'y_true': y_values, 
        'y_pred_xgb_rf': oof_predictions,
        'smiles': smiles_for_oof.values 
    }

print("\n=== Cross-validation Results Summary ===")
for target, score in mae_scores.items():
    print(f"{target}: {score:.4f}")


Training for Tg...
Data shape: (16590, 303), features: 303
  Fold 1/5
    Fold 1 MAE: 15.0686
  Fold 2/5
    Fold 2 MAE: 14.8264
  Fold 3/5
    Fold 3 MAE: 14.4393
  Fold 4/5
    Fold 4 MAE: 14.9533
  Fold 5/5
    Fold 5 MAE: 15.1358
  CV MAE for Tg: 14.8847 ± 0.2463

Training for FFV...
Data shape: (15784, 377), features: 377
  Fold 1/5
    Fold 1 MAE: 0.0054
  Fold 2/5
    Fold 2 MAE: 0.0054
  Fold 3/5
    Fold 3 MAE: 0.0057
  Fold 4/5
    Fold 4 MAE: 0.0055
  Fold 5/5
    Fold 5 MAE: 0.0055
  CV MAE for FFV: 0.0055 ± 0.0001

Training for Tc...
Data shape: (1732, 301), features: 301
  Fold 1/5
    Fold 1 MAE: 0.0179
  Fold 2/5
    Fold 2 MAE: 0.0162
  Fold 3/5
    Fold 3 MAE: 0.0183
  Fold 4/5
    Fold 4 MAE: 0.0180
  Fold 5/5
    Fold 5 MAE: 0.0201
  CV MAE for Tc: 0.0181 ± 0.0012

Training for Density...
Data shape: (2594, 380), features: 380
  Fold 1/5
    Fold 1 MAE: 0.0178
  Fold 2/5
    Fold 2 MAE: 0.0185
  Fold 3/5
    Fold 3 MAE: 0.0161
  Fold 4/5
    Fold 4 MAE: 0.0187
  Fo

In [11]:
print("--- Generating GNN predictions on TEST data ---")
gnn_test_predictions_df = run_gnn_predictions(test_smiles, test_ids)
print("\n GNN Test Predictions Complete:")
print(gnn_test_predictions_df.head())
print("\n--- Generating GNN predictions on ALL TRAINING data ---")
all_training_smiles = set()
for target in TARGET_VARIABLES:
    all_training_smiles.update(oof_data[target]['smiles'])

unique_train_smiles_list = list(all_training_smiles)
unique_train_ids = list(range(len(unique_train_smiles_list)))
print(f"Found {len(unique_train_smiles_list)} unique training SMILES to predict.")

gnn_train_preds_df = run_gnn_predictions(unique_train_smiles_list, unique_train_ids)
gnn_train_preds_df['SMILES'] = unique_train_smiles_list

print("\n GNN Training Predictions Complete.")

--- Generating GNN predictions on TEST data ---
\n==================== Processing GNN for label: Tg ====================
Loaded 10 models for Tg ensemble.
\n==================== Processing GNN for label: FFV ====================
Loaded 10 models for FFV ensemble.
\n==================== Processing GNN for label: Tc ====================
Loaded 10 models for Tc ensemble.
\n==================== Processing GNN for label: Density ====================
Loaded 10 models for Density ensemble.
\n==================== Processing GNN for label: Rg ====================
Loaded 10 models for Rg ensemble.

 GNN Test Predictions Complete:
           id          Tg       FFV        Tc   Density         Rg
0  1109053969  189.751847  0.373679  0.187229  1.168659  21.798934
1  1422188626  174.595003  0.375173  0.266134  1.103980  23.134463
2  2032016830  111.029739  0.350787  0.250217  1.113623  19.598631

--- Generating GNN predictions on ALL TRAINING data ---
Found 30132 unique training SMILES to predict.


# Meta-model training and inference

In [12]:
final_predictions_df = pd.DataFrame({'id': test_ids})
meta_model_scores = {}

gnn_train_preds_lookup = gnn_train_preds_df.set_index('SMILES')

for target in TARGET_VARIABLES:
    print(f"--- Stacking models for {target} ---")
    

    oof_df = pd.DataFrame({
        'SMILES': oof_data[target]['smiles'],
        'y_true': oof_data[target]['y_true'],
        'y_pred_xgb_rf': oof_data[target]['y_pred_xgb_rf']
    })
    

    oof_df = oof_df.merge(gnn_train_preds_lookup[[target]], on='SMILES', how='left')
    oof_df.rename(columns={target: 'y_pred_gnn'}, inplace=True)
    

    oof_aligned = oof_df.dropna()

    X_meta_train = oof_aligned[['y_pred_xgb_rf', 'y_pred_gnn']].values
    y_meta_train = oof_aligned['y_true'].values


    meta_model = XGBRegressor(n_estimators=100, max_depth=3, learning_rate=0.1, random_state=RANDOM_STATE)
    meta_model.fit(X_meta_train, y_meta_train)
    

    meta_oof_preds = meta_model.predict(X_meta_train)
    meta_mae = mean_absolute_error(y_meta_train, meta_oof_preds)
    meta_model_scores[target] = meta_mae
    print(f"  Meta-Model MAE on OOF data for {target}: {meta_mae:.4f}")
    

    test_pred_xgb_rf = predictions_df[target].values
    test_pred_gnn = gnn_test_predictions_df[target].values
    X_meta_test = np.vstack([test_pred_xgb_rf, test_pred_gnn]).T
    

    final_predictions_df[target] = meta_model.predict(X_meta_test)


print("\n=== Meta-Model OOF MAE Summary ===")
print(pd.DataFrame.from_dict(meta_model_scores, orient='index', columns=['Meta_Model_MAE']))

final_predictions_df.to_csv('submission.csv', index=False)
print("\nFinal ensembled submission saved to submission.csv")

print("\n--- Final Submission Preview ---")
print(final_predictions_df.head())

--- Stacking models for Tg ---
  Meta-Model MAE on OOF data for Tg: 13.8022
--- Stacking models for FFV ---
  Meta-Model MAE on OOF data for FFV: 0.0027
--- Stacking models for Tc ---
  Meta-Model MAE on OOF data for Tc: 0.0148
--- Stacking models for Density ---
  Meta-Model MAE on OOF data for Density: 0.0159
--- Stacking models for Rg ---
  Meta-Model MAE on OOF data for Rg: 0.4977

=== Meta-Model OOF MAE Summary ===
         Meta_Model_MAE
Tg            13.802226
FFV            0.002717
Tc             0.014811
Density        0.015938
Rg             0.497692

Final ensembled submission saved to submission.csv

--- Final Submission Preview ---
           id          Tg       FFV        Tc   Density         Rg
0  1109053969  164.702530  0.374093  0.181062  1.164261  20.206488
1  1422188626  154.549393  0.375053  0.268137  1.112596  19.997940
2  2032016830  143.726624  0.350086  0.241577  1.112596  19.832661
